In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from pathlib import Path
import os
from IPython.display import display

__author__ = "Gifrey John M. Sulay"
__copyright__ = "Xavier University - Engineering Resource Center"

In [2]:
import datetime
from dateutil.relativedelta import relativedelta

def set_time_range():
    run = True
    while run:
        try:
            input_start_date = str(input("Please input Start Date (Format=Month,Date,Year): "))
            start_date_datetime = pd.to_datetime(input_start_date,format="%m,%d,%Y")

            input_end_date = str(input("Please input End Date (Format=Month,Date,Year): "))
            end_date_datetime = pd.to_datetime(input_end_date,format="%m,%d,%Y")

            print((f"Date start is {start_date_datetime} and End Date is {end_date_datetime}"))
            date_confirm = input("Type y if the input is correct or n to try again: ")
            if date_confirm == "y":
                start_date = start_date_datetime
                end_date = end_date_datetime + datetime.timedelta(days = 1)
                run = False
            elif date_confirm == "n":
                continue
            else:
                raise ValueError
                
        except Exception as e:
            print(e)
    return start_date, end_date

is_range = input("Do you wish to select a range? (y for yes, n for no and select entire available data)")

run=True
while run:
    try:
        if is_range == "y" or "Y":
            start_date, end_date = set_time_range()
        elif is_range == "n" or "N":
            start_date, end_date = (None, None)
        else:
            print('Not valid argument. Please try again')
            raise ValueError
        run = False
    except Exception as e:
        print(e)
    

In [3]:
cag_path = Path("Edited_Data/Cagayan/cag.csv")
rg_path = Path("Edited_Data/Cagayan/rg1.csv")

cag = pd.read_csv(cag_path)[['Timestamp', 'Sensor Value']]
rg = pd.read_csv(rg_path)[['Timestamp', 'Sensor Value']]

display('Cagayan Water Table',cag.head(),cag.tail())
display('Rain Gauge Data',rg.head(),rg.tail())

In [4]:
#Delete duplicates
cag_1 = cag.drop_duplicates(keep='first')
rg_1 = rg.drop_duplicates(keep='first')

cag_dropped=len(cag)-len(cag_1)
rg_dropped=len(rg)-len(rg_1)

print(f"No of cells dropped from water level table is {cag_dropped} cells")
print(f"No of cells dropped from rain gauge table is {rg_dropped} cells")

cag = cag_1
rg = rg_1

display(cag,rg)

In [5]:
def column_apply(df,column, function):
    df[column] = df[column].apply(function)

def map_apply(df,function):
    df =  df.applymap(function)

In [6]:
import datetime

column_apply(cag,'Timestamp', lambda x: pd.to_datetime(x).replace(second = 0, minute=0))
column_apply(rg,'Timestamp', lambda x: pd.to_datetime(x).replace(second = 0, minute=0))

# column_apply(cag,'Timestamp', lambda x: x.replace(second = 0, minute=0))
# column_apply(rg,'Timestamp', lambda x: x.replace(second = 0, minute=0))

In [7]:
rg = rg.groupby(by='Timestamp').mean().reset_index()
cag = cag.groupby(by='Timestamp').mean().reset_index()

In [8]:
cag=cag.rename(columns={"Sensor Value":"Water Level"})
rg=rg.rename(columns={"Sensor Value":"Rain Gauge"})

display(cag,rg)

In [9]:
def make_base_time(cag,rg):
    rg_start_date = rg.iloc[0,0]
    rg_end_date = rg.iloc[-1,0]
    cag_start_date = cag.iloc[0,0]
    cag_end_date = cag.iloc[-1,0]


    if rg_start_date > cag_start_date:
        start_date = cag_start_date
    else:
        start_date = rg_start_date

    if rg_end_date < cag_end_date:
        end_date = cag_end_date
    else:
        end_date = rg_end_date

    base_time = pd.DataFrame({'Timestamp':pd.date_range(start=start_date, end=end_date, freq="H")})
    return base_time

if start_date != None:
    base_time = pd.DataFrame({'Timestamp':pd.date_range(start=start_date, end=end_date, freq="H")})
else:
    base_time = make_base_time(cag,rg)

In [10]:
cag = pd.merge(base_time, cag, how='left', on='Timestamp')
main_df = pd.merge(cag,rg,how='left', on='Timestamp')

In [11]:
missing_water = main_df.loc[main_df['Water Level'].isna()]
missing_rg = main_df.loc[main_df['Rain Gauge'].isna()]

display(missing_water)
display(missing_rg)

In [12]:
main_df['Hour'] = main_df['Timestamp'].apply(lambda x: pd.to_datetime(x).strftime('%H'))

#Rearranges the datafrom to place the Hour data as the first column
main_df = main_df[['Hour']+main_df.columns.values[:-1].tolist()]

In [13]:
main_df['Rain Gauge'] = main_df['Rain Gauge'].astype('float64').interpolate()

In [14]:
def rg_diff(lst):
    new_list = [0]
    count = 1
    for value in lst[1:]:
        if value-lst[count-1] > 0:
            value-=lst[count-1]
            new_list.append(value)
        else:
            new_list.append(0)
        count += 1
    return new_list

main_df['RG_Diff'] = rg_diff(main_df.loc[:,'Rain Gauge'])

main_df

In [15]:
def clean_water_level(df, column, diff_limit = 0.5):
    corrected_df = df.copy()
    water_level = corrected_df[column]
    #diff limit determines whether the difference is erroneous if difference is more than diff limit then data is erroneous
    is_erroneous=[0]
    count = 1
    
    for i in water_level[1:]:
        val = i-water_level[count - 1]
        val = abs(val)
        if val >= diff_limit:
            is_erroneous.append(1)
        else:
            is_erroneous.append(0)
        count += 1
    corrected_df['err_check']=is_erroneous
    corrected_df.loc[corrected_df['err_check']==1, column] = np.nan
    
    edited_water_level=corrected_df.loc[:, column]
    final_water_level=edited_water_level.astype('float64').interpolate()
    corrected_df=corrected_df.drop(columns=[column,'err_check'])
    corrected_df[column]=final_water_level
    return corrected_df

corrected_main_df = clean_water_level(main_df, 'Water Level', diff_limit=0.5)

In [16]:
display('Raw Water Table',main_df,'Corrected Water Table',corrected_main_df)

In [17]:
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

fig,ax = plt.subplots(1,2, figsize=(25,8), sharey=True)

water_level = main_df['Water Level'].to_numpy()
ax[0].plot(main_df['Timestamp'], water_level)
ax[0].set_title('Raw Water Table')
water_level_max_index = argrelextrema(water_level, np.greater , order = 10)
ax[0].scatter(main_df.loc[water_level_max_index[0], 'Timestamp'], water_level[water_level_max_index[0]], color='r')

for index in water_level_max_index[0]:
    timestamp=str(main_df.loc[index, 'Timestamp'])
    ax[0].annotate("{}\n{}\n{:.2f}m".format(timestamp.split()[0],timestamp.split()[1], water_level[index]),
    (main_df.loc[index, 'Timestamp'], water_level[index]+0.05),
    ha='center')

corrected_water_level = corrected_main_df['Water Level'].to_numpy()
ax[1].plot(main_df['Timestamp'], corrected_water_level)
ax[1].set_title('Corrected Water Table')
corrected_water_level_max_index = argrelextrema(corrected_water_level, np.greater , order = 10)
ax[1].scatter(main_df.loc[corrected_water_level_max_index[0], 'Timestamp'], corrected_water_level[corrected_water_level_max_index[0]], color='r')

for index in corrected_water_level_max_index[0]:
    timestamp=str(main_df.loc[index, 'Timestamp'])
    ax[1].annotate("{}\n{}\n{:.2f}m".format(timestamp.split()[0],timestamp.split()[1],corrected_water_level[index]),
    (main_df.loc[index, 'Timestamp'], corrected_water_level[index] + 0.05,),
    ha='center')

label = np.arange(0,len(main_df['Timestamp']),1) % 200 == 0
for x in ax:
    x.set_xticks(main_df["Timestamp"].loc[label])
    x.set_xlabel("Timestamp")
    x.set_ylabel("Water Level in Meters")
    x.grid(visible=True)

    x.spines['top'].set_visible(False)
    x.spines['right'].set_visible(False)
    x.spines['bottom'].set_visible(False)
    x.spines['left'].set_visible(False)
fig.tight_layout()

water_fig = fig
plt.show()

In [18]:
fig, ax = plt.subplots(2,1, figsize=(22,8), sharex=True, sharey=True)

ax[0].plot(main_df['Timestamp'], main_df['Rain Gauge'])
ax[0].set_title('Rain Gauge Reading')

raw_rg_arry = main_df['Rain Gauge'].to_numpy()
raw_rg_max_index = argrelextrema(raw_rg_arry, np.greater , order = 10)[0]
ax[0].scatter(main_df['Timestamp'][raw_rg_max_index], raw_rg_arry[raw_rg_max_index], color='r')

for index in raw_rg_max_index:
    timestamp = str(main_df['Timestamp'][index])
    ax[0].annotate("{}\n{}\n{:.2f}".format(timestamp.split()[0],timestamp.split()[1],raw_rg_arry[index]),
    (main_df['Timestamp'][index],raw_rg_arry[index] + 0.05),
    va='bottom',ha='center',fontsize='x-small')

ax[1].plot(main_df['Timestamp'], main_df['RG_Diff'])
ax[1].set_title('Rain Gauge Difference')

rg_diff_arry = main_df['RG_Diff'].to_numpy()
rg_diff_max_index = argrelextrema(raw_rg_arry, np.greater , order = 10)[0]
ax[1].scatter(main_df['Timestamp'][rg_diff_max_index], rg_diff_arry[rg_diff_max_index], color='r')

for index in rg_diff_max_index:
    timestamp = str(main_df['Timestamp'][index])
    ax[1].annotate("{}\n{}\n{:.2f}".format(timestamp.split()[0],timestamp.split()[1],rg_diff_arry[index]),
    (main_df['Timestamp'][index],rg_diff_arry[index] + 0.05),
    va='bottom',ha='center',fontsize='x-small')

label = np.arange(0,len(main_df['Timestamp']),1) % 200 == 0
for x in ax:
    x.set_xticks(main_df["Timestamp"].loc[label])
    x.set_xlabel("Timestamp")
    x.set_ylabel("Water Level in Meters")
    x.grid(visible=True)

    x.spines['top'].set_visible(False)
    x.spines['right'].set_visible(False)
    x.spines['bottom'].set_visible(False)
    x.spines['left'].set_visible(False)

fig.tight_layout()

rg_fig = fig
plt.show()

In [19]:
time_frmt = '%Y/%m/%d %H:00:00'
main_df['Timestamp'] = main_df['Timestamp'].apply(lambda x: x.strftime(time_frmt))

In [20]:
main_df['Corrected Water Level'] = corrected_main_df['Water Level']

start_str = start_date.strftime("%Y%d%m")
end_str = end_date.strftime("%Y%d%m")

filename = f"{start_str}_{end_str} Report"
parent_directory = "Cleaned_Cagayan/"

directory = os.path.join(parent_directory,filename)
print("Creating directory {}".format(directory))
try:
    os.mkdir(directory)
except:
    print("Appending to Existing Directory")

with pd.ExcelWriter(os.path.join(directory,f"{filename}.xlsx")) as writer:
    main_df.to_excel(writer, sheet_name=f"{filename}")

water_fig.savefig(os.path.join(directory,f"Water Level {filename}.png"),transparent=False,dpi=600)
rg_fig.savefig(os.path.join(directory,f"Rain Gaugy {filename}.png"),transparent=False,dpi=600)

print("Excel Sheet and Figures are successfully saved")

In [21]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook -e "$notebook_dir"

notebook_dir = "".join(r'.\', notebook_dir)
!jupyter nbconvert --to html  ""

In [22]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook -e "$notebook_dir"

notebook_dir = "".join(r'.\ '.strip(), notebook_dir)
!jupyter nbconvert --to html  "$notebook_dir"

In [23]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook -e "$notebook_dir"

notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
!jupyter nbconvert --to html  "$notebook_dir"

In [24]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook -e "$notebook_dir"

notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
import subprocess

subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"')

In [25]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook -e "$notebook_dir"

notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
import subprocess

subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"', shell=True)

In [26]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook "$notebook_dir"

notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
import subprocess

subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"', shell=True)

In [27]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook "$notebook_dir"

# notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
# import subprocess

# subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"', shell=True)

In [28]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook "$notebook_dir"

# notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
# import subprocess

# subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"', shell=True)

In [29]:
notebook_dir = os.path.join(directory,f"{filename}.ipynb")
%notebook "$notebook_dir"

# notebook_dir = "".join([r'.\ '.strip(), notebook_dir])
# import subprocess

# subprocess.call(f'jupyter nbconvert --to html  "{notebook_dir}"', shell=True)